In [6]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import resnet50, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model


In [4]:
train_datagen = ImageDataGenerator(preprocessing_function= resnet50.preprocess_input)

In [5]:
val_datagen = ImageDataGenerator()

In [ ]:
# funzioni e importazione
COMMON_PATH = '/home/marco/Scrivania/Progetti_vari/fototrappola/foto_rete/images2'
samples = ['train', 'val']
class_types = ['lupi', 'orsi', 'other']


def graying_and_saving(original_path, new_path):
    original_img = cv2.imread(original_path)
    gray_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    gray_img = np.stack((gray_img,)*3, axis=-1)
    cv2.imwrite(new_path, gray_img)

for sample in samples:
    samples_path = os.path.join(COMMON_PATH, sample)
    
    for class_type in class_types:
        class_path = os.path.join(samples_path, class_type)
        
        for i in os.listdir(class_path):
            original_path = os.path.join(class_path, i)
            new_img_name = 'gray-' + i
            new_path = os.path.join(class_path, new_img_name)
            graying_and_saving(original_path, new_path)
print(f'finito!')

In [10]:
train_generator = train_datagen.flow_from_directory(COMMON_PATH+'/train',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=30,
                                                    class_mode='categorical')
val_generator = val_datagen.flow_from_directory(COMMON_PATH+'/val',
                                                target_size=(224,224))

Found 360 images belonging to 3 classes.
Found 176 images belonging to 3 classes.


In [11]:
raw_xtest, ytest = next(val_generator)

In [12]:
xtest = resnet50.preprocess_input(raw_xtest)

In [14]:
base_model = ResNet50(weights='imagenet', include_top=False)

2025-01-07 11:38:09.279270: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [15]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
pred = Dense (3, activation='softmax')(x)

In [16]:
transfer_model = Model(inputs=base_model.input, outputs=pred)

freeze

In [18]:
len(base_model.layers)

175

In [25]:
for i,layer in enumerate(base_model.layers):
    print(f'{i},{layer.name}')

0,input_layer
1,conv1_pad
2,conv1_conv
3,conv1_bn
4,conv1_relu
5,pool1_pad
6,pool1_pool
7,conv2_block1_1_conv
8,conv2_block1_1_bn
9,conv2_block1_1_relu
10,conv2_block1_2_conv
11,conv2_block1_2_bn
12,conv2_block1_2_relu
13,conv2_block1_0_conv
14,conv2_block1_3_conv
15,conv2_block1_0_bn
16,conv2_block1_3_bn
17,conv2_block1_add
18,conv2_block1_out
19,conv2_block2_1_conv
20,conv2_block2_1_bn
21,conv2_block2_1_relu
22,conv2_block2_2_conv
23,conv2_block2_2_bn
24,conv2_block2_2_relu
25,conv2_block2_3_conv
26,conv2_block2_3_bn
27,conv2_block2_add
28,conv2_block2_out
29,conv2_block3_1_conv
30,conv2_block3_1_bn
31,conv2_block3_1_relu
32,conv2_block3_2_conv
33,conv2_block3_2_bn
34,conv2_block3_2_relu
35,conv2_block3_3_conv
36,conv2_block3_3_bn
37,conv2_block3_add
38,conv2_block3_out
39,conv3_block1_1_conv
40,conv3_block1_1_bn
41,conv3_block1_1_relu
42,conv3_block1_2_conv
43,conv3_block1_2_bn
44,conv3_block1_2_relu
45,conv3_block1_0_conv
46,conv3_block1_3_conv
47,conv3_block1_0_bn
48,conv3_block1_

In [20]:
for layer in transfer_model.layers[:174]:
    layer.trainable = False
for layer in transfer_model.layers[174:]:
    layer.trainable = True

compile

In [27]:
transfer_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy']
    )

fit

In [30]:
%%time
history = transfer_model.fit(train_generator,
                             steps_per_epoch=train_generator.n // train_generator.batch_size,
                             validation_data=(xtest, ytest),
                             epochs=2)

/home/marco/anaconda3/envs/tensenv2/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2


2025-01-07 11:59:25.899153: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 96337920 exceeds 10% of free system memory.
2025-01-07 11:59:26.145393: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 99809280 exceeds 10% of free system memory.
2025-01-07 11:59:26.240809: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 96337920 exceeds 10% of free system memory.
2025-01-07 11:59:26.397431: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 96337920 exceeds 10% of free system memory.
2025-01-07 11:59:26.637946: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 96337920 exceeds 10% of free system memory.


12/12 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - accuracy: 0.5018 - loss: 1.8670 - val_accuracy: 0.8438 - val_loss: 0.4411
Epoch 2/2
12/12 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 0.9368 - loss: 0.2052 - val_accuracy: 1.0000 - val_loss: 0.0730
CPU times: user 7min 12s, sys: 31 s, total: 7min 43s
Wall time: 1min 27s
